# Trading EDA

To install conda and binance packages to this notebook uncomment the code below

In [1]:
#%conda install -c plotly plotly=5.9.0
#%conda install pip
#%conda install twisted
%pip install plotly==5.9.0
%pip install twisted
%pip install binance-connector==1.13.0
%pip install pandas
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Init binance client

In [2]:
from binance.spot import Spot as Client
import os
import sys
import yaml

# Load 
def load_config():
    """
    Load config from cfg folder respecting the order: defaults, app.yaml, environment vars
    """
    # Defaults
    cfg_dir="./../biml/cfg"
    default_cfg_path = f"{cfg_dir}/app-defaults.yaml"
    with open(default_cfg_path, "r") as appdefaults:
        config = yaml.safe_load(appdefaults)

    # Custom config, should contain custom information,
    cfg_path = f"{cfg_dir}/app.yaml"
    if os.path.exists(cfg_path):
        with open(cfg_path) as app:
            config.update(yaml.safe_load(app))
    else:
        sys.exit(
            f"Config {cfg_path} not found. Please copy cfg/app-defaults.yaml to {cfg_path} "
            f"and update connection info there.")

    # Enviroment variabless
    config.update(os.environ)
    return config


# Create binance client
config=load_config()
key,secret,url=config["biml.connector.key"],config["biml.connector.secret"],config["biml.connector.url"]
client = Client(key=key, secret=secret, base_url=url, timeout=10)


Get account info

In [3]:
account=client.account()
print(account)


{'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'canTrade': True, 'canWithdraw': False, 'canDeposit': False, 'brokered': False, 'updateTime': 1664040497230, 'accountType': 'SPOT', 'balances': [{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '1.00500000', 'locked': '0.00000000'}, {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '9904.31459000', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}], 'permissions': ['SPOT']}


Get orders from binance

In [71]:
# Get orders
import pandas as pd
symbol="BTCUSDT"

# Get orders, but market orders have price=0
orders=pd.DataFrame(client.get_orders(symbol))
if not orders.empty: orders=orders.astype({'time': 'datetime64[ms]','updateTime':'datetime64[ms]','price':'float64','origQty':'float64'}).set_index('time',drop=False)

# Get trades and pull orders' prices from there
trades=pd.DataFrame( client.my_trades(symbol))
if not trades.empty: trades=trades.astype({'time': 'datetime64[ms]', 'price':'float64'}).set_index('time',drop=False)    
orders=orders.drop("price",axis=1).merge(trades[["orderId","price"]],on="orderId")

# Calculate profit of closing orders
orders["delta_price"]=orders["price"].diff()
orders["profit"] = orders["delta_price"]*orders["origQty"]
orders.loc[~orders["type"].isin(["LIMIT_MAKER", "STOP_LOSS_LIMIT"]),["profit"]]=np.nan

orders.set_index("time", drop=False, inplace=True)
orders.tail()


,symbol,orderId,orderListId,clientOrderId,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,origQuoteOrderQty,price,delta_price,profit
time,,,,,,,,,,,,,,,,,,,,
2022-09-24 17:18:53.163,BTCUSDT,6928621,14758,efUsLHpO085tO6OOUoekOj,0.001,0.00100000,19.13122000,FILLED,GTC,LIMIT_MAKER,SELL,0.00000000,0.00000000,2022-09-24 17:18:53.163,2022-09-24 17:21:39.209,True,0.00000000,19131.22,1.40,0.00140
2022-09-24 17:26:58.847,BTCUSDT,6930970,-1,pCe0MQ6mZ43A7ebt6hnQCJ,0.001,0.00100000,19.13002000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2022-09-24 17:26:58.847,2022-09-24 17:26:58.847,True,0.00000000,19130.02,-1.20,NaN
2022-09-24 17:26:59.174,BTCUSDT,6930972,14759,F7fTrdaMDGfthHelBeqBaL,0.001,0.00100000,19.13199000,FILLED,GTC,LIMIT_MAKER,SELL,0.00000000,0.00000000,2022-09-24 17:26:59.174,2022-09-24 17:27:22.230,True,0.00000000,19131.99,1.97,0.00197
2022-09-24 17:28:05.106,BTCUSDT,6931261,-1,MCshgHd0ynqBQFPo1KDk31,0.001,0.00100000,19.12558000,FILLED,GTC,MARKET,BUY,0.00000000,0.00000000,2022-09-24 17:28:05.106,2022-09-24 17:28:05.106,True,0.00000000,19125.58,-6.41,NaN
2022-09-24 17:28:05.923,BTCUSDT,6931267,14760,e0mLeFwu4BjS3Cptb5pYGw,0.001,0.00100000,19.12641000,FILLED,GTC,LIMIT_MAKER,SELL,0.00000000,0.00000000,2022-09-24 17:28:05.923,2022-09-24 17:28:17.230,True,0.00000000,19126.41,0.83,0.00083


Look at opened orders

In [6]:
orders_opened=client.get_open_orders(symbol)
print(orders_opened)

[]


Get candles from local history starting from earliest trade date ending by latest trade date.

In [7]:
import os
import glob
import pandas as pd
from datetime import timedelta,date


def read_candles(start_date_inc, end_date_inc):
    """ Read days from start to end from local history"""
    symbol="BTCUSDT"    
    data_dir=f"./../data/{symbol}"
    print(f"start date:{start_date_inc}, end date:{end_date_inc}, all inclusive")
    candles = pd.DataFrame()
    cur_candles_list=[]
    for n in range(int ((end_date_inc - start_date_inc).days)+1):
        cur_date = start_date_inc + timedelta(n)    
        csv_path=f"{data_dir}/{cur_date}_{symbol}_1m.csv"     
        if not os.path.exists(csv_path): continue
        #print(f"Read {cur_date} candles from {csv_path}")
        cur_candles=pd.read_csv(csv_path,index_col="close_time")[["open","high","low","close","vol"]]
        cur_candles_list.append(cur_candles)
    candles = pd.concat(cur_candles_list)
    return candles

def clean_candles(df):
    """ Remove outliers """
    q_low = df["low"].quantile(0.01)
    q_high  = df["high"].quantile(0.99)
    df=df[df["high"]< q_high]
    df=df[df["low"] > q_low ]
    return df

# Read candles for last 3 days
last_date=orders.index.max().date()
first_date=last_date - timedelta(days=0)

candles = read_candles(first_date, last_date)
candles = clean_candles(candles)
candles.tail()


start date:2022-09-24, end date:2022-09-24, all inclusive


,open,high,low,close,vol
close_time,,,,,
2022-09-24 23:42:59.999,18932.05,18941.38,18931.65,18938.16,5.013635
2022-09-24 23:43:59.999,18941.22,18949.15,18941.22,18943.44,6.276134
2022-09-24 23:44:59.999,18943.23,18951.05,18943.23,18950.17,5.087081
2022-09-24 23:45:59.999,18946.00,18949.99,18938.23,18938.94,7.021751
2022-09-24 23:46:59.999,18939.23,18939.62,18927.63,18930.48,3.809458


## EDA

### Look at trades quantity cumulative sum

## Look at trades on candlestick chart

In [76]:
# Install a conda package in the current Jupyter kernel
#%conda install -c plotly plotly=5.9.0
# Draw candlechart
import plotly as py
from plotly import graph_objects as go
import pandas as pd

df=candles.copy()
#df=clean_candles(df)
fig = go.Figure(data=[ \
                    go.Candlestick(\
                        x=df.index,\
                        open=df['open'],\
                        high=df['high'],\
                        low=df['low'],\
                        close=df['close'])
                     ])

# Customize ay to distribute captions by y so they do not overlap by x
ay,aymax,aystep=0,50,10
# Annotate buy, sell on the chart
for traderec in orders[(botorders["status"] == "FILLED") & (botorders.index > df.index.min())].iterrows():
    ay=(ay+aystep)%aymax
    trade=traderec[1]
    color='green' if trade['side']=="BUY" else 'red'
    text_short='buy' if trade['side']=="BUY"  else 'sell'
    status= f", profit: {round(trade['profit'], 10)}" if ~np.isnan(trade['profit']) else ""
    #text=f"{trade['time']} {text_short} at {str(trade['price'])}, profit: {str(trade['profit'])}"
    text=f"{trade['time']} {text_short} at {str(trade['price'])} {status}"
    fig.add_annotation(x=trade['time'], \
                       y=trade['price'],\
                       text=text_short,\
                       hovertext=text,\
                       showarrow=True,arrowhead=5,arrowcolor=color,ay=-ay,arrowwidth=2,
                       font={'color':color}
                      )
fig.show()


## Profit

In [9]:
#import matplotlib.pyplot as plt
import plotly.express as px
# Calc profit, considering only closing trades
profit=botorders[(botorders.status=="FILLED")&(botorders["profit"])].copy()
profit["cum_profit"] = profit["profit"].cumsum()

#Plot
fig = px.line(profit, x="time", y="profit")
fig.show()

In [10]:
import matplotlib.pyplot as plt
import plotly.express as px
# Calc profit, considering only closing trades
profit=botorders[(botorders.status=="FILLED")&(botorders["profit"])].copy()
#profit["profit"]=profit["price"].diff()*(profit["qty"]/profit["qty"])
profit["cum_profit"] = profit["profit"].cumsum()

#Plot
fig = px.line(profit, y="cum_profit")
fig.show()

## Look at the account

In [11]:
print(account)

{'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'canTrade': True, 'canWithdraw': False, 'canDeposit': False, 'brokered': False, 'updateTime': 1664040497230, 'accountType': 'SPOT', 'balances': [{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '1.00500000', 'locked': '0.00000000'}, {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '9904.31459000', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}], 'permissions': ['SPOT']}
